In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")

nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")
nlp("Je teste spacy à Paris")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Device set to use cpu


[{'entity_group': 'LOC',
  'score': np.float32(0.9953166),
  'word': 'Paris',
  'start': 16,
  'end': 22}]

### Traiter un exemple de texte

In [ ]:
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

# Charger le modèle et le tokenizer pré-entrainés
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")
nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Initialiser les listes pour les tokens et les entités nommées
liste_token = []
liste_entitesnom = []
liste_candidat = []

# Lire le texte depuis un fichier
with open("sample_data/Corpus/APPOLINAIRE_Caligrammes.txt", "r", encoding="utf-8") as f:
    texte = f.read().strip()

# Utiliser le tokenizer pour découper le texte en tokens
encoded = tokenizer(texte)
liste_token = encoded.tokens()

# Effectuer la reconnaissance d'entités nommées avec CamemBERT
resultats = nlp(texte)
liste_entitesnom = [{"mot": entite['word'], "type": entite['entity_group']} for entite in resultats] # Créer une liste des entités nommées extraites (avec leur type)


# Extraire tous les mots d'entités (en s'assurant qu'ils correspondent avec les tokens)
entite_words = set(entite['mot'] for entite in liste_entitesnom)
# Supprimer des caractères(<s>, ▁)
valid_tokens = [tok for tok in liste_token if tok not in ['<s>', '</s>', '▁', ':', 's', '']]
# Filtrer les tokens qui ne sont pas des entités
non_entite_tokens = [tok.lstrip('▁') for tok in valid_tokens if tok not in entite_words] # retirer le marqueur BPE ▁
liste_candidat = [tok for tok in non_entite_tokens if tok != ':'] # retirer les symboles comme ':'”



# Afficher les résultats
print(f"Tokens        : {len(liste_token)}")
print(f"Entités       : {len(liste_entitesnom)}")
print(f"Candidats   : {len(liste_candidat)}")
print("-" * 20)
print("Ex_tokens:", liste_token[:20])
print("Ex_entités:", liste_entitesnom[:20])
print("Ex_candidat:", liste_candidat[:20])




Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (21297 > 512). Running this sequence through the model will result in indexing errors


Tokens        : 21297
Entités       : 23
Candidats   : 20562
--------------------
Ex_tokens: ['<s>', '▁Rappel', '▁de', '▁votre', '▁demande', ':', '▁Format', '▁de', '▁téléchargement', ':', '▁:', '▁Texte', '▁Vue', 's', '▁1', '▁à', '▁26', '2', '▁sur', '▁26']
Ex_entités: [{'mot': 'Calligrammes', 'type': 'MISC'}, {'mot': 'Guillaume Apollinaire', 'type': 'PER'}, {'mot': 'La Fresnaye', 'type': 'PER'}, {'mot': 'Apollinaire', 'type': 'PER'}, {'mot': 'Guillaume', 'type': 'PER'}, {'mot': 'Lausanne', 'type': 'LOC'}, {'mot': 'La Fresnaye', 'type': 'PER'}, {'mot': 'Roger de', 'type': 'PER'}, {'mot': 'Français', 'type': 'MISC'}, {'mot': 'Calligrammes', 'type': 'MISC'}, {'mot': 'Collection du Bouquet', 'type': 'MISC'}, {'mot': 'Collection : Collection du Bouquet', 'type': 'MISC'}, {'mot': 'Bibliothèque nationale de France', 'type': 'LOC'}, {'mot': 'Bibliothèque nationale de France', 'type': 'LOC'}, {'mot': 'Dessins de', 'type': 'MISC'}, {'mot': 'La Fresnaye', 'type': 'PER'}, {'mot': 'GU', 'type': 'MIS

In [ ]:
import json
with open("entites_nommes_APPOLINAIRE_Caligrammes.json", "w", encoding="utf-8") as json_file:
    json.dump(liste_entitesnom, json_file, ensure_ascii=False, indent=4)

### Traiter au niveau du corpus

In [ ]:
import re, glob
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline


tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")
nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")

liste_token = []
liste_entitesnom = []
liste_candidat = []


for path_fichier in glob.glob("sample_data/Corpus/*.txt"):
    with open(path_fichier, "r", encoding="utf-8") as f:
        texte = f.read().strip()

        encoded = tokenizer(texte)
        liste_token.extend(encoded.tokens())

        resultats = nlp(texte)
        for entite in resultats:
            liste_entitesnom.append((entite['word'], entite['entity_group']))


        entite_words = set(entite[0] for entite in liste_entitesnom)
        valid_tokens = [tok for tok in liste_token if tok not in ['<s>', '</s>', '▁', 's', '']]
        non_entite_tokens_premier_filtrage = [tok.lstrip('▁') for tok in valid_tokens if tok not in entite_words]
        non_entite_tokens = [tok for tok in non_entite_tokens_premier_filtrage if re.match(r'[a-zA-ZÀ-ÿ]+', tok)]
        liste_candidat.extend(list(set(non_entite_tokens)))


print(f"Total des tokens : {len(liste_token)}")
print(f"Total des entités : {len(liste_entitesnom)}")
print(f"Total des candidats : {len(liste_candidat)}")
print("-" * 20)
print("Ex_tokens:", liste_token[:20])
print("Ex_entités:", liste_entitesnom[:20])
print("Ex_candidats:", liste_candidat[:20])



Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (24761 > 512). Running this sequence through the model will result in indexing errors


Total des tokens : 375172
Total des entités : 225
Total des candidats : 92176
--------------------
Ex_tokens: ['<s>', '▁Rappel', '▁de', '▁votre', '▁demande', ':', '▁Format', '▁de', '▁téléchargement', ':', '▁:', '▁Texte', '▁Vue', 's', '▁1', '▁à', '▁1', '36', '▁sur', '▁1']
Ex_entités: [('Fleurs d', 'MISC'), ('avril : poésies', 'MISC'), ('Jeanne Loiseau', 'PER'), ('Loiseau', 'PER'), ('Jeanne', 'PER'), ('Paris', 'LOC'), ('Français', 'MISC'), ('Bibliothèque nationale de France', 'LOC'), ('Bibliothèque nationale de France', 'LOC'), ('JEANNE LOISEAU', 'PER'), ('Fleurs d’Avril POESIES A Fleurs d ’ Av r', 'MISC'), ('JEANNE LOISEAU', 'PER'), ('Fleurs cTAvril m', 'MISC'), ('POESIES PARIS ALPHONSE', 'MISC'), ('LEMERRE', 'PER'), ('CHOISEUL', 'MISC'), ('Fleurs (TAvril PROLOGUE', 'MISC'), ('Fleurs ci’ Avril', 'MISC'), ('f Mon Livre Bleu', 'MISC'), ('Mon Livre Bleu', 'MISC')]
Ex_candidats: ['aujourd', 'poids', 'Eu', 'tres', 'une', 'catalogue', 'soirée', 'dépasse', 'pha', 'avions', 'patient', 'royal', 

In [ ]:
import json
with open("tous_entites_nommes_Camembert.json", "w", encoding="utf-8") as json_file:
    json.dump(liste_entitesnom, json_file, ensure_ascii=False, indent=4)

### Ttaiter au niveau de chaque texte

In [ ]:
import re
import glob
import json
import os
import time
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from tqdm import tqdm



tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")
nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")


# Liste pour stocker les données de résumé
summary_data = []

# Chemin du dossier Corpus

corpus_path = "sample_data/Corpus/"

# Parcourir tous les fichiers .txt dans le dossier Corpus
for path_fichier in glob.glob(os.path.join(corpus_path, "*.txt")):
    start_time = time.time()
    with open(path_fichier, "r", encoding="utf-8") as f:
        texte = f.read().strip()

        # Initialiser les listes pour chaque fichier
        liste_token = []
        liste_entitesnom = []
        liste_candidat = []

        encoded = tokenizer(texte)
        liste_token = encoded.tokens()

        # Stocker les entites du fichier actuel
        resultats = nlp(texte)
        for entite in resultats:
            liste_entitesnom.append((entite['word'], entite['entity_group']))  # (mot, catégorie d'entité)

        # Stocker les candidats du fichier actuel
        entite_words = set(entite[0] for entite in liste_entitesnom)
        valid_tokens = [tok for tok in liste_token if tok not in ['<s>', '</s>', '▁', 's', '']]
        non_entite_tokens_premier_filtrage = [tok.lstrip('▁') for tok in valid_tokens if tok not in entite_words]
        non_entite_tokens = [tok for tok in non_entite_tokens_premier_filtrage if re.match(r'[a-zA-ZÀ-ÿ]+', tok)]
        liste_candidat = non_entite_tokens

        # Nom du fichier actuel
        file_name = os.path.basename(path_fichier)

        # Créer le répertoire pour stocker les résultats du fichier
        output_dir = os.path.join("Camembert_output", file_name)
        os.makedirs(output_dir, exist_ok=True)

        # Sauvegarder les résultats (entités et mots candidats) dans un fichier JSON
        output_data = {
            "entites_nom": liste_entitesnom,
            "candidats": liste_candidat
        }

        # Chemin du fichier JSON pour chaque fichier de texte
        json_file_path = os.path.join(output_dir, f"{file_name}_entites_candidats.json")
        with open(json_file_path, "w", encoding="utf-8") as json_file:
            json.dump(output_data, json_file, indent=4, ensure_ascii=False)

        # Afficher le nombre d'entités et de mots candidats dans la console
        print(f"Fichier : {file_name}")
        print(f"Tokens  : {len(liste_token)}")
        print(f"Entités : {len(liste_entitesnom)}")
        print(f"Candidats: {len(liste_candidat)}")
        print("-" * 20)
        print(f"  20 premières entités : {liste_entitesnom[:20]}")
        print(f"  20 premiers candidats : {liste_candidat[:20]}")
        print("=" * 40)


        # Sauvegarder les entités et les mots candidats dans des fichiers texte
        with open(os.path.join(output_dir, f"{file_name}_entites.txt"), "w", encoding="utf-8") as entites_file:
            for entite in liste_entitesnom:
                entites_file.write(f"{entite[0]} - {entite[1]}\n")

        with open(os.path.join(output_dir, f"{file_name}_candidats.txt"), "w", encoding="utf-8") as candidats_file:
            for mot in liste_candidat:
                candidats_file.write(f"{mot}\n")
        # Ajouter les données de résumé pour chaque fichier dans la liste
        summary_data.append({
            "Fichier": file_name,
            "Entites_Nom": len(liste_entitesnom),
            "Mots_Candidats": len(liste_candidat),
            "Temps_Procédé (s)": round(time.time() - start_time, 2)  # calculer le temps de traitement de chaque texte
        })
# Convertir les données de résumé en DataFrame
summary_df = pd.DataFrame(summary_data)

# Afficher le tableau résumé directement dans la console
print("\nRésumé des résultats des fichiers traités :\n")
print(summary_df)

# Afficher un message une fois que tous les fichiers ont été traités
print("\nTraitement terminé pour tous les fichiers.")


Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (24761 > 512). Running this sequence through the model will result in indexing errors


Fichier : LOISEAU_Fleurs-d-avril.txt
Tokens  : 24761
Entités : 20
Candidats: 19271
--------------------
  20 premières entités : [('Fleurs d', 'MISC'), ('avril : poésies', 'MISC'), ('Jeanne Loiseau', 'PER'), ('Loiseau', 'PER'), ('Jeanne', 'PER'), ('Paris', 'LOC'), ('Français', 'MISC'), ('Bibliothèque nationale de France', 'LOC'), ('Bibliothèque nationale de France', 'LOC'), ('JEANNE LOISEAU', 'PER'), ('Fleurs d’Avril POESIES A Fleurs d ’ Av r', 'MISC'), ('JEANNE LOISEAU', 'PER'), ('Fleurs cTAvril m', 'MISC'), ('POESIES PARIS ALPHONSE', 'MISC'), ('LEMERRE', 'PER'), ('CHOISEUL', 'MISC'), ('Fleurs (TAvril PROLOGUE', 'MISC'), ('Fleurs ci’ Avril', 'MISC'), ('f Mon Livre Bleu', 'MISC'), ('Mon Livre Bleu', 'MISC')]
  20 premiers candidats : ['Rappel', 'de', 'votre', 'demande', 'Format', 'de', 'téléchargement', 'Texte', 'Vue', 'à', 'sur', 'Nombre', 'de', 'pages', 'Notice', 'complète', 'Titre', 'Fleurs', 'd', 'avril']
Fichier : RIMBAUD_Illuminations-et-Une-saison-en-enfer-et-Notice-Paul-Verlain

### Exclure l'erreur due à la limitation du nombre de tokens traitée par Camembert

In [ ]:
# Vérifer le longueur maximale des tokens supportée par le modèle
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
print(tokenizer.model_max_length)


512



Utilisation de la méthode de fenêtre glissante pour traiter des textes très longs. La fenêtre glissante consiste à diviser un long texte en plusieurs petits morceaux, chaque morceau contenant au maximum 512 tokens, puis à traiter chaque morceau séparément.（使用滑动窗口 方法处理超长文本。滑动窗口是通过分割长文本为多个小段，每个小段最多 512 个 tokens，然后对每个小段分别进行处理。）



Fonction sliding_window : Divise un texte long en plusieurs morceaux, chaque morceau contenant au maximum max_len tokens (par défaut 512), et le pas de glissement est défini par step (par défaut 256).（sliding_window 函数：将长文本分割为多个 chunks，每个 chunk 最多包含 max_len 个 tokens（默认为 512），并且每次滑动的步长是 step（默认为 256）。）

In [ ]:
# Méthode de fenêtre glissante
import concurrent.futures
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

#Initialiser le modèle
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")
nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")
def sliding_window(tokenizer, text, max_len=512, step=256):
    tokens = tokenizer.encode(text)
    chunks = []

    # Diviser le texte en plusieurs morceaux, chaque morceau contient au maximum max_len tokens
    for i in range(0, len(tokens), step):
        chunk = tokens[i:i + max_len]
        chunks.append(chunk)

    return chunks

# Effectuer la reconnaissance des entités pour chaque morceau
def process_chunk(chunk):
    chunk_text = tokenizer.decode(chunk)
    return nlp(chunk_text)

# Lire le texte et utiliser la méthode de fenêtre glissante pour traiter les textes longs
with open("sample_data/Corpus/APPOLINAIRE_Caligrammes.txt", "r", encoding="utf-8") as f:
    text = f.read().strip()

chunks = sliding_window(tokenizer, text, max_len=512, step=256)

# Utiliser tqdm pour afficher la barre de progression et mesurer le temps
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Affichage de la barre de progression avec tqdm
    results = list(tqdm(executor.map(process_chunk, chunks), total=len(chunks), desc="Traitement des morceaux", unit="morceau"))

# Afficher le nombre de morceaux générés et le nombre de résultats d'entités
print("=" * 40)
print(f"Nombre de morceaux générés : {len(chunks)}")
print(f"Nombre d'EN générés : {len(results)}")
print(results)

Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (21297 > 512). Running this sequence through the model will result in indexing errors
Traitement des morceaux: 100%|██████████| 84/84 [02:45<00:00,  1.97s/morceau]

Nombre de morceaux générés : 84
Nombre d'EN générés : 84
[[{'entity_group': 'MISC', 'score': np.float32(0.9828121), 'word': 'Calligrammes', 'start': 129, 'end': 142}, {'entity_group': 'PER', 'score': np.float32(0.9982582), 'word': 'Guillaume Apollinaire', 'start': 144, 'end': 166}, {'entity_group': 'PER', 'score': np.float32(0.99716437), 'word': 'La Fresnaye', 'start': 179, 'end': 191}, {'entity_group': 'PER', 'score': np.float32(0.9985141), 'word': 'Apollinaire', 'start': 200, 'end': 212}, {'entity_group': 'PER', 'score': np.float32(0.9925296), 'word': 'Guillaume', 'start': 213, 'end': 223}, {'entity_group': 'LOC', 'score': np.float32(0.9935164), 'word': 'Lausanne', 'start': 264, 'end': 272}, {'entity_group': 'PER', 'score': np.float32(0.9972395), 'word': 'La Fresnaye', 'start': 310, 'end': 322}, {'entity_group': 'PER', 'score': np.float32(0.8177874), 'word': 'Roger de', 'start': 323, 'end': 332}, {'entity_group': 'MISC', 'score': np.float32(0.6305047), 'word': 'Français', 'start': 54

In [ ]:
EN = [{"mot": entite['word'], "type": entite['entity_group']} for entite in resultats]
print(EN)

[{'mot': 'Derniers vers', 'type': 'MISC'}, {'mot': 'Comtesse de Noailles', 'type': 'PER'}, {'mot': 'Noailles', 'type': 'PER'}, {'mot': 'Anna de', 'type': 'PER'}, {'mot': 'Public do', 'type': 'MISC'}, {'mot': 'Bibliothèque nationale de France', 'type': 'LOC'}, {'mot': 'Bibliothèque nationale de France', 'type': 'LOC'}, {'mot': 'TESSE DE NOAILLES', 'type': 'LOC'}, {'mot': 'GRASSE', 'type': 'LOC'}, {'mot': 'T', 'type': 'PER'}, {'mot': 'DE NOAILLES', 'type': 'LOC'}, {'mot': 'GRASSET', 'type': 'LOC'}, {'mot': 'ANNE-JULES & HÉLÈNE', 'type': 'PER'}, {'mot': 'Constantin PHOTIADÈS', 'type': 'PER'}, {'mot': 'comtesse de Noailles', 'type': 'PER'}, {'mot': 'Poème de P amour', 'type': 'MISC'}, {'mot': 'V Honneur de souffrir', 'type': 'MISC'}, {'mot': 'Souvenirs d^un jardin d^Angleterre^', 'type': 'MISC'}, {'mot': 'Vivants et des morts', 'type': 'MISC'}, {'mot': 'Un soir à Londres', 'type': 'MISC'}, {'mot': 'Forces éternelles', 'type': 'MISC'}, {'mot': 'Revue de Paris', 'type': 'MISC'}, {'mot': 'Bon

In [ ]:
import json
with open("modifier_entites_nommes_APPOLINAIRE_Caligrammes.json", "w", encoding="utf-8") as json_file:
    json.dump(EN, json_file, ensure_ascii=False, indent=4)

#### La quantité de tokens traitée par Camembert est limitée, mais cela n'a pas entraîné un traitement partiel du texte en raison des paramètres par défaut.